In [ ]:
import json 
from typing import List,Dict,Tuple
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

GEMMA = "gemma3:1b"
DEEPSEEK = "deepseek-r1:1.5b"

# LLM Question Generator
Agente para generar preguntas y respuestas utilizando LLM. Este agente usa prompts estructurados para generar preguntas específicas. Además, crea respuestas para cada pregunta generada. 

In [9]:
question_prompt = """ 
Genera {n_questions} preguntas sobre el tema "{topic}".
Las preguntas deben ser:
- Claras y específicas
- De diferentes niveles de dificultad (básico, intermedio, avanzado)
- Educativas y relevantes al tema

Formato de respuesta (JSON):
{{
  "preguntas": [
    "pregunta 1",
    "pregunta 2",
    ...
  ]
}}
"""
answer_prompt = """ 
Responde la siguiente pregunta sobre {topic} de manera clara, precisa y educativa.
La respuesta debe ser informativa pero concisa (2-4 párrafos máximo).

Pregunta: {question}

Respuesta:
"""

In [10]:
class QAGenerator:
  def __init__(self, model_name:str=GEMMA, temperature:float=0.7):
    """_summary_

    Args:
        model_name (str, optional): _description_. Defaults to GEMMA.
        temperature (float, optional): _description_. Defaults to 0.7.
    """
    self.llm = OllamaLLM(model=model_name, temperature=temperature)
    self.q_prompt = ChatPromptTemplate.from_template(question_prompt)
    self.a_prompt = ChatPromptTemplate.from_template(answer_prompt)
  
  def generate_questions(self, topic:str, n_questions:int, attempts:int=4) -> List[str]:
    "Genera preguntas para un tema específico" 
    try:
      chain = self.q_prompt | self.llm
      response = chain.invoke({
        "topic": topic,
        "n_questions": n_questions
      })
      
      # Intentar parsear JSON
      try:
        result = json.loads(response)
        questions = result.get("preguntas", [])
      except json.JSONDecodeError:
        # Si falla el JSON, extraer preguntas manualmente
        lines = response.split('\n')
        questions = []
        for line in lines:
          line = line.strip()
          if line and ('?' in line or line.startswith(('-', '*', '1.', '2.'))):
            # Limpiar formato de lista
            clean_question = line.replace('-', '').replace('*', '').strip()
            if clean_question.startswith(tuple('123456789')):
              clean_question = '. '.join(clean_question.split('. ')[1:])
            if clean_question:
              questions.append(clean_question)
            
      # Asegurar que tenemos el número correcto de preguntas
      if len(questions) > n_questions:
        questions = questions[:n_questions]
      elif len(questions) < n_questions:
        # Generar preguntas adicionales si es necesario
        additional_needed = n_questions - len(questions)
        for i in range(additional_needed):
          questions.append(f"Pregunta adicional sobre {topic} #{i+1}")
            
      return questions
            
    except Exception as e:
      if attempts != 0: self.generate_questions(topic, n_questions, attempts-1)
      print(f"Error generando preguntas para {topic}: {e}")
      # Preguntas de respaldo
      fallback_questions = [
        f"¿Qué es {topic}?",
        f"¿Cuáles son las características principales de {topic}?",
        f"¿Cómo se aplica {topic} en la práctica?",
        f"¿Cuáles son los conceptos fundamentales de {topic}?",
        f"¿Qué importancia tiene {topic} en la actualidad?"
      ]
      return fallback_questions[:n_questions]
  
  def generate_answer(self, question:str, topic:str) -> str:
    "Genera respuesta para una pregunta específica"
    try:
      chain = self.a_prompt | self.llm 
      response = chain.invoke({
        "question": question,
        "topic": topic
      })
      return response.strip()
    except Exception as e:
      return f"Error generando respuesta: {e}"
  
  def __call__(self, topics:Dict[str,int], verbose:bool=False) -> Dict[str, List[Tuple[str,str]]]:
    """Genera conjunto completo de preguntas y respuestas

    Args:
        topics (Dict[str,int]): diccionario con formato {"tema": cantidad_de_preguntas}

    Returns:
        Dict[str, List[Tuple[str,str]]]: diccionario con formato {"tema": [(pregunta, respuesta), ...]}
    """
    QA_set = {}
    for topic,n_questions in topics.items():
      # generar preguntas
      if verbose: print(f"=> Generando {n_questions} preguntas para: {topic}")
      questions = self.generate_questions(topic, n_questions)
      
      # generar respuestas
      qa_pairs = []
      for i, question in enumerate(questions, 1):
        if verbose: print(f"==> Generando respuesta {i}/{len(questions)}")
        answer = self.generate_answer(question, topic)
        qa_pairs.append((question, answer))
      
      QA_set[topic] = qa_pairs
    return QA_set

In [11]:
topics = {
  "Inteligencia Artificial": 3,
  "Matemática Discreta": 2,
}
generator = QAGenerator()
generator(topics=topics)

{'Inteligencia Artificial': [('"pregunta": "Considerando los desafíos éticos asociados al desarrollo y despliegue de la IA (como sesgo algorítmico y pérdida de empleos), ¿qué estrategias proponen los investigadores para mitigar estos riesgos y asegurar un uso responsable de la IA? Justifica tus respuestas con ejemplos de posibles enfoques.",',
   'La ética de la Inteligencia Artificial (IA) se ha convertido en un campo de preocupación creciente, impulsado por los desafíos que plantea el desarrollo y la implementación de sistemas de IA. Uno de los principales desafíos es el **sesgo algorítmico**, que ocurre cuando los sistemas de IA aprenden de datos que reflejan sesgos existentes en la sociedad. Esto puede llevar a resultados injustos o discriminatorios en áreas como la contratación, la justicia penal, o incluso la concesión de préstamos. Por ejemplo, un sistema de reconocimiento facial entrenado principalmente con imágenes de personas de una raza específica podría tener dificultades p